<a href="https://colab.research.google.com/github/asteinig4018/BirdWatcher/blob/master/makeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make a Model
The purpose of this notebook is to create a TFlite model for Raspberry Pi to detect birds (ideally in an image and not individually). I'll try to use the Caltech set. 

In [ ]:
#in progress: gather requirements
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_datasets as tfds

In [ ]:
#idk check the version
print(tf.__version__)

# Build the Dataset

## Downloading the dataset:

In [ ]:
bird_train_data = tfds.load('caltech_birds2011', split='train', shuffle_files=True)
bird_test_data = tfds.load('caltech_birds2011', split='test', shuffle_files=False)

In [10]:
#physicall download
!gdown https://drive.google.com/u/0/uc?id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45

ERROR: unknown command "intall" - maybe you meant "install"
Downloading...
From: https://drive.google.com/u/0/uc?id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45
To: /content/CUB_200_2011.tgz
100% 1.15G/1.15G [00:13<00:00, 84.9MB/s]
tar (child): v: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [12]:
!tar -xzf /content/CUB_200_2011.tgz

## Prepare the data
Grab mobilenet's preprocess function:

In [ ]:
preprocess_input = tf.keras.applications.mobilenet.preprocess_input

# Get YOLOv5 ver 6 
??

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt
import torch
from IPython.display import Image, clear_output # for showing images

# Get YOLOv4 Tiny
Getting a pretrained model will be half the battle. We'll start by grabbing a maintained (by RoboFlow supposedly and idk how long for) for of darknet. 

In [ ]:
%mkdir content
%cd /content
!git clone https://github.com/roboflow-ai/darknet.git

In [ ]:
%cd /content/darknet

In [ ]:
# Straight from RoboFlow's Colab guide:
os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

def getGPUArch(argument):
  try:
    argument = argument.strip()
    # All Colab GPUs
    archTypes = {
        "Tesla V100-SXM2-16GB": "-gencode arch=compute_70,code=[sm_70,compute_70]",
        "Tesla K80": "-gencode arch=compute_37,code=sm_37",
        "Tesla T4": "-gencode arch=compute_75,code=[sm_75,compute_75]",
        "Tesla P40": "-gencode arch=compute_61,code=sm_61",
        "Tesla P4": "-gencode arch=compute_61,code=sm_61",
        "Tesla P100-PCIE-16GB": "-gencode arch=compute_60,code=sm_60"

      }
    return archTypes[argument]
  except KeyError:
    return "GPU must be added to GPU Commands"
os.environ['ARCH_VALUE'] = getGPUArch(os.environ['GPU_TYPE'])

print("GPU Type: " + os.environ['GPU_TYPE'])
print("ARCH Value: " + os.environ['ARCH_VALUE'])

In [ ]:
# Continuing with the copying:
#install environment from the Makefile
#note if you are on Colab Pro this works on a P100 GPU
#if you are on Colab free, you may need to change the Makefile for the K80 GPU
#this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
!make

In [ ]:
# Also copied and not so sure about how "newly" these are
#download the newly released yolov4-tiny weights
%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29